In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import ChebConv  # Using Chebyshev convolution
from torch_geometric.nn import knn_graph
from torch_geometric.utils import get_laplacian, get_mesh_laplacian
# from torch_sparse import coalesce
from torch_geometric.utils import degree

from embedder_dataset import EmbedderDataset
from torch.utils.data import DataLoader

from models import *
from misc import compute_edge_indices, compute_laplacian#, compute_batched_edge_indices
from point_cloud_datasest import PointCloudDataset
torch.__version__, "cuda" if torch.cuda.is_available() else "cpu"

('2.4.0+cu124', 'cuda')

In [2]:
# point_cloud = torch.randn(1000, 3)  
# point_cloud.shape, point_cloud.size(0)
# edge_index = knn_graph(point_cloud, k=6, loop=False)

# # Ensure the Laplacian is computed based on the correct edge index
# num_nodes = point_cloud.size(0)
# laplacian = compute_laplacian(edge_index, num_nodes)
# print(f"Laplacian shape: {laplacian.shape}")


In [3]:
device = "cpu" if not torch.cuda.is_available() else "cuda"

# ds_dir = "datasets_embedded"
# ds_fname = "ds_1024.pth" # 1024 points per shape
ds_path = r"C:\Projects\multiSetup\datasets_embedded\ds_1024.pth"
ds = torch.load(ds_path)
dataset_data = ds.data

print(f"Using {device} device, ds length - {len(ds)}, data.shape - {dataset_data.shape}")

EPOCHS = 200
BATCH_SIZE = 128
SPLIT_FACTOR = .8
TRAIN_MODE = (False, True)[1]
REMOVE_OLD_MODELS = True
POINTS_PER_SHAPE = 1024
POINT_DIM = 3


C:\Users\5010858\AppData\Local\Temp/ipykernel_30696/2273564222.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ds = torch.load(ds_path)


Using cuda device, ds length - 27571, data.shape - (27571, 3072)


In [4]:
diffision_ae_ds = PointCloudDataset(torch.from_numpy(dataset_data), k=6)
len(diffision_ae_ds)
train_loader = DataLoader(diffision_ae_ds, batch_size=BATCH_SIZE, shuffle=True)

In [5]:
# batch = next(iter(train_loader))
# x, edge_index, laplacian = batch
# laplacian

In [7]:
def train_autoencoder(model, train_loader, device, epochs=1000, lr=1e-3):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.MSELoss()
    for epoch in range(epochs):
        for batch in train_loader:
            X, edge_indices, laplacians = batch
            model.train()
            optimizer.zero_grad()

            # X = X.view(BATCH_SIZE, POINTS_PER_SHAPE, POINT_DIM).to(device)
            X = X.to(device) # torch.Size([128, 1024, 3])
            edge_indices = edge_indices.to(device) # torch.Size([128, 2, 6144])
            laplacians = laplacians.to(device) # torch.Size([128, 6144]) надо [128, 6144]
            
            # Forward pass
            reconstructed = model(X, edge_indices, None)
            loss = loss_fn(reconstructed, X)

            # Backward pass
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch}, Loss: {loss.item()}')
        if epoch % 10 == 0:
            torch.save(model.state_dict(), f"models/{epoch}_state_dict.pth")

hidden_features = 64
latent_dim = 32

model = DiffusionNetAutoencoder(POINT_DIM, hidden_features, latent_dim).to(device)

train_autoencoder(model, train_loader, device)

Error on build_exception_info_response.
Traceback (most recent call last):
  File "c:\Python39\lib\site-packages\debugpy\_vendored\pydevd\_pydevd_bundle\pydevd_comm.py", line 1476, in build_exception_info_response
    line_text = linecache.getline(original_filename, lineno)
  File "c:\Python39\lib\linecache.py", line 30, in getline
    lines = getlines(filename, module_globals)
  File "c:\Python39\lib\linecache.py", line 46, in getlines
    return updatecache(filename, module_globals)
  File "c:\Python39\lib\linecache.py", line 137, in updatecache
    lines = fp.readlines()
  File "c:\Python39\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xed in position 637: invalid continuation byte


RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 128 but got size 2 for tensor number 1 in the list.

In [8]:
"get_laplacian: IndexError: The shape of the mask [2, 6144] at index 0 does not match the shape of the indexed tensor [128, 2, 7168] at index 0"
"norm_laplacian: IndexError: The shape of the mask [2, 6144] at index 0 does not match the shape of the indexed tensor [128, 1024, 1024] at index 0"
6144/1024, 7168/1024

(6.0, 7.0)